Đây là script chạy explainer cho các mô hình khác nhau gồm các bước:
- Clone repository từ GitHub
- Load dữ liệu đã qua xử lý từ thư mục data/DATA_PROCESSED (Các fle csv này được lưu sau khi chạy script dataflow2026_hd4k_run_model.ipynb)
- Load weight mô hinh đã huấn luyện từ thư mục data/weights
- Chạy phân tích bằng SHAP, LIME, và DiCE rồi in ra ảnh minh họa

**Lưu ý**:
- Script này **không thể** chạy local. Chúng tôi đã đề xuất hai phương án thay thế kèm hướng dẫn để chạy là chạy trên [Google Colab](./HOW_TO_RUN_COLAB.md) và chạy trên [Kaggle](./HOW_TO_RUN_KAGGLE.md).
- Đây là notebook chạy chung cho **12 trường hợp** của dự án trustee (3 hướng tiếp cận x 4 mô hình). Thời gian chạy notebook tối đa là 40 phút (với mọi trường hợp).
- Có hai cell code cần lưu ý là cell 2 và cell 7:
    - Cell 2: code thay đổi tùy theo bạn chạy trên Google Colab hay Kaggle
    - Cell 7: hai biến MODEL_NAME và APPROACH_TYPE thay đổi tùy theo hướng tiếp cận và mô hình bạn chọn

In [ ]:
!git clone https://github.com/CryAndRRich/trustee.git

In [ ]:
# Nếu chạy trên Google Colab, uncomment 5 dòng dưới
# %cd /content/trustee
# TRUSTEE_PATH = "/content/trustee"
# !unzip "/content/data_for_scripts.zip" -d "/content/data_for_scripts"
# DATA_ROOT = "/content/data_for_scripts/DATA_PROCESSED"
# WEIGHT_ROOT = "/content/data_for_scripts/weights"

# Nếu chạy trên Kaggle, uncomment 4 dòng dưới
# %cd /kaggle/working/trustee
# TRUSTEE_PATH = "/kaggle/working/trustee"
# DATA_ROOT = "/kaggle/input/df2026-hd4k/DATA_PROCESSED"
# WEIGHT_ROOT = "/kaggle/input/df2026-hd4k/weights"

In [ ]:
!pip install -r requirements.txt

In [3]:
import sys

if TRUSTEE_PATH not in sys.path:
    sys.path.append(TRUSTEE_PATH)

In [4]:
from utils import *
from explainer import *

In [5]:
TRAIN_FRESH = f"{DATA_ROOT}/train_fresh.csv"
TRAIN_SENIOR = f"{DATA_ROOT}/train_senior.csv"

VAL_FRESH = f"{DATA_ROOT}/val_fresh.csv"
VAL_SENIOR = f"{DATA_ROOT}/val_senior.csv"

In [6]:
# MODEL_NAME là "Decision_Tree", "Random_Forest", "XGBoost" hoặc "LightGBM"
MODEL_NAME = "LightGBM"
# EXT là "joblib", "json" hoặc "txt" tùy theo MODEL_NAME
map_ext = {
    "Decision_Tree": "joblib",
    "Random_Forest": "joblib",
    "XGBoost": "json",
    "LightGBM": "txt"
}
EXT = map_ext[MODEL_NAME]

# APPROACH_TYPE là "Credits", "Gap" hoặc "Ratio"
APPROACH_TYPE = "Credits"

MODEL_FRESH = f"{WEIGHT_ROOT}/{MODEL_NAME.lower()}/{APPROACH_TYPE.lower()}_prediction_{MODEL_NAME.lower()}_fresher.{EXT}"
MODEL_SENIOR = f"{WEIGHT_ROOT}/{MODEL_NAME.lower()}/{APPROACH_TYPE.lower()}_prediction_{MODEL_NAME.lower()}_senior.{EXT}"

In [7]:
feats_senior = [
    "TC_DANGKY", "SEMESTER_INDEX", "SV_NAM_THU",
    
    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    
    "R2_AVG_GPA", "R2_SUM_FAIL", "R2_PASS_RATE",
    "FAIL_TREND_R2", "GPA_TREND_R2",
    
    "R3_AVG_GPA", "R3_SUM_FAIL",
    "PRESSURE_VS_R2", "PRESSURE_VS_R3", "OVERLOAD_R3",
    
    "TOTAL_EARNED", "OVERLOAD_VS_MAX",
    "HIST_AVG_GPA", "HIST_MAX_PASSED", "HIST_MAX_GPA", "HIST_STD_GPA",    
]

feats_fresh = [
    "TC_DANGKY", "SEMESTER_INDEX", "PTXT", "TOHOP_XT",
    
    "DIEM_TRUNGTUYEN", "DIEM_CHUAN", 
    "SCORE_GAP", "ENTRY_RANK", "BENCHMARK_TIER",
    "Z_SCORE", "GAP_RATIO",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    "PRESSURE_VS_R2" 
]

In [ ]:
# Phân tích toàn cục bằng SHAP
# (hàm explain_model_shap được định nghĩa trong trong explainer/shap_explainer.py)
df_res_fresh, exp_fresh = explain_model_shap(
    model_path=MODEL_FRESH, 
    data_path=VAL_FRESH, 
    feats=feats_fresh,
    approach_type=APPROACH_TYPE
)

In [ ]:
df_res_senior, exp_senior = explain_model_shap(
    model_path=MODEL_SENIOR, 
    data_path=VAL_SENIOR, 
    feats=feats_senior,
    approach_type=APPROACH_TYPE
)

In [ ]:
# Phân tích cục bộ bằng LIME
# (hàm explain_model_lime được định nghĩa trong trong explainer/lime_explainer.py)
errors_fresh = explain_model_lime(
    model_path=MODEL_FRESH, 
    train_path=TRAIN_FRESH,
    val_path=VAL_FRESH, 
    feats=feats_fresh,
    approach_type=APPROACH_TYPE
)

In [ ]:
errors_senior = explain_model_lime(
    model_path=MODEL_SENIOR, 
    train_path=TRAIN_SENIOR,
    val_path=VAL_SENIOR, 
    feats=feats_senior,
    approach_type=APPROACH_TYPE
)

In [ ]:
# Phân tích phản chứng bằng DiCE
# (hàm explain_model_dice được định nghĩa trong trong explainer/dice_explainer.py)
dice_exp_senior = explain_model_dice(
    model_path=MODEL_SENIOR, 
    train_path=TRAIN_SENIOR,
    val_path=VAL_SENIOR, 
    feats=feats_senior,
    approach_type=APPROACH_TYPE
)